In [1]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import json
#from scipy import stats
#import scipy.interpolate as interp
from pprint import pprint
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14

In [2]:
import FEMmodule as FEM

# Toy FEM - python version

This code is the Python replica of the matlab code used in *LAB01* of the course **Nonlinear Analysis of Aerospace Structures** held by Marco Morandini, Riccardo Vescovini at *Politecnico di Milano*

## Build matrices

- *nodes* matrix: containing nodes coordinates
- *elements* matrix: connectivity matrix associating nodes to elements
- *constrained elements* vector: list of constrained (fixed) nodes

Parameters are **nx** and **ny**: number of elements in *x* and *y* directions.

Default plate size is l=10, w=2. It can be changes passing additional parameters to **Nodes**

In [9]:
nx = 2
ny = 2
l = 10.
w = 2.

In [10]:
nodes, elements, constr_nodes = FEM.Nodes(nx,ny)
#nodes, elements, constr_nodes = FEM.Nodes(nx,ny, l, w)

Create a Gaussian integrator for **2D** 2$\times$2 matrix

In [6]:
GI = FEM.GaussIntegr2D2x2

Create a *plane stress* **Elastic Tensor** with parameters **E** and **$\nu$**

In [7]:
E = FEM.PlaneStressElasticTensor(1.e4, 0.3)

Create a set of *bilinear weights*, used to interppolate a **2D** function on a $(x,y)$ grid

In [11]:
W = FEM.BilinearWeights

Build vectors of *constrained* and *free* **DOFs**

In [17]:
constr_dofs = np.append(constr_nodes, constr_nodes + nodes.shape[0])
free_dofs = np.array([i for i in range(1,2*nodes.shape[0]+1) if i not in constr_dofs])

Build matrices

In [18]:
ne = nodes.shape[0]*nodes.shape[1]
GlobalStiff = np.zeros((ne,ne))
GlobalRes = np.zeros(ne)
GlobalDispl = np.zeros_like(GlobalRes)

**Force** vector

In [19]:
force = np.array([0. ,1.])